#### Topic Modeling on Article data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import shared_data

In [4]:
content = shared_data.SharedData().get_content()

In [ ]:
content.head()

Remove all docs that are not in English

In [ ]:
content.language.value_counts()

In [ ]:
content.shape

In [ ]:
content = content[content['language'] == 'en']

In [ ]:
content.shape

In [ ]:
no_dups = content.sort_values('event_timestamp').drop_duplicates(subset=['title', 'text_description'], keep='last')

In [ ]:
no_dups.head()

In [ ]:
no_dups.reset_index(inplace=True)

In [ ]:
no_dups.interaction_type.value_counts()

In [ ]:
no_dups[no_dups['title'] == "Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's"]

In [ ]:
content[content['title'] == "Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's"]

Makes sense

In [ ]:
content = no_dups

Drop everything except the text.

In [ ]:
trimmed = content[['title', 'text_description', 'interaction_type']]

In [ ]:
trimmed.head()

Preprocessing

In [ ]:
trimmed['text_no_punc'] = trimmed.text_description.str.replace(r'[^\w\s]+', '')

In [ ]:
trimmed.head()

In [ ]:
# Convert trimmed.text_no_punc to lowercase and assign to trimmed.text_no_punc_lower
trimmed['text_no_punc_lower'] = trimmed.text_no_punc.str.lower()

In [ ]:
trimmed.head()

#### Build a WordCloud

In [ ]:
%pip install wordcloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
# Create a wordcloud of trimmed.text_no_punc_lower
long_string = ','.join(trimmed.text_no_punc_lower)
wc = WordCloud(background_color="white", max_words=5000, contour_width=5, contour_color='steelblue')
wc.generate(long_string)
wc.to_image()

#### Prepare the data for LDA

In [ ]:
%pip install gensim

In [ ]:
import gensim
from gensim.utils import simple_preprocess

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
stopwords_en = stopwords.words('english')

In [ ]:
# Define a function to convert sentences to words using gensim's simple_preprocess
def sentences_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True))

# define a function to remove stopwords
# def remove_stopwords(words):
    # return [w for w in words if w not in stopwords_en]

In [ ]:
# Define a function to remove stopwords using gensim's simple_preprocess
def remove_stopwords(docs):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords_en] for doc in docs]

In [ ]:
data = trimmed.text_description.values.tolist()
data_words = list(sentences_to_words(data))

In [ ]:
# Create bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Define a function to make bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [ ]:
%pip install spacy

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Define a function to lemmatize given texts
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
# data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops)

In [ ]:
# sentences = trimmed.text_description.values.tolist()

In [ ]:
# words_array = list(sentences_to_words(sentences))

In [ ]:
# words_array[0]

In [ ]:
# no_stopwords = [remove_stopwords(words) for words in words_array]

In [ ]:
# no_stopwords[0]

Build a Dictionary

In [ ]:
from gensim.corpora import Dictionary

data_dictionary = Dictionary(data_lemmatized)
corpus = [data_dictionary.doc2bow(word_arr) for word_arr in data_lemmatized]

In [ ]:
# data_dictionary.doc2bow(words[0])
# len(words[0])
# words[0][13]

In [ ]:
# corpus[0]

In [ ]:
# num_topics = 20

In [ ]:
update_every = [1, 2, 5, 10]
chunksize = [10, 20, 25, 50, 75, 100]
passes = [1, 2, 5, 10]
num_topics = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [ ]:
import itertools

In [ ]:
# Create a DataFrame to store the results
results = pd.DataFrame(columns=['update_every', 'chunksize', 'passes', 'num_topics', 'perplexity', 'coherence'])

In [ ]:
for x in itertools.product(update_every, chunksize, passes, num_topics):
    (ue, cs, p, nt) = x
    lda_model = gensim.models.ldamodel.LdaModel(
        corpus=corpus, 
        id2word=data_dictionary, 
        random_state=42, 
        alpha='auto', 
        eta='auto',
        num_topics=nt, 
        update_every=ue, 
        chunksize=cs, 
        passes=p, 
        per_word_topics=True
    )
    
    # Compute Perplexity
    perplexity = lda_model.log_perplexity(corpus)

    # Compute Coherence Score
    coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=data_dictionary, coherence='c_v')
    coherence = coherence_model_lda.get_coherence()

    results = results.append({'update_every': ue, 'chunksize': cs, 'passes': p, 'num_topics': nt, 'perplexity': perplexity, 'coherence': coherence}, ignore_index=True)

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus, 
    id2word=data_dictionary, 
    num_topics=num_topics, 
    random_state=42, 
    update_every=1, 
    chunksize=50, 
    passes=10, 
    alpha='auto', 
    eta='auto',
    per_word_topics=True
)

In [ ]:
# from pprint import pprint
# pprint(lda_model.print_topics())
lda_model.print_topics()

In [ ]:
doc_lda = lda_model[corpus]

Evaluation

In [ ]:
lda_model.log_perplexity(corpus)

In [ ]:
doc_lda

In [ ]:
doc_lda[0]

In [ ]:
lda_model.get_document_topics(corpus[0], minimum_probability=0.01)

In [ ]:
lda_model.show_topic(0)

In [ ]:
all_topics = lda_model.get_document_topics(corpus, minimum_probability=0.01)

In [ ]:
all_topics_matrix = gensim.matutils.corpus2csc(all_topics)

In [ ]:
all_topics_matrix

Matrix is 10 X 3026, meaning it has one row for each topic, and one column for each item.

So take the transpose

In [ ]:
all_topics_arr = all_topics_matrix.T.toarray()

In [ ]:
all_topics_df = pd.DataFrame(all_topics_arr)

In [ ]:
all_topics_df.head()

In [ ]:
all_topics_df[1].value_counts()

Squashing the topics into a single list

In [ ]:
for idx, row in enumerate(lda_model[corpus]):
    print(idx, row)
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    print(row)

    # Extract the first item from each tuple in row
    topic_indices = [x[0] for x in row]

    # print(lda_model.show_topic(row[0][0]))
    print(topic_indices)
    break

In [ ]:
def get_topic_indices(row):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    topic_indices = [x[0] for x in row]
    return topic_indices

In [ ]:
# Map the get_topic_indices function to lda_model[corpus]
topic_indices = list(map(get_topic_indices, lda_model[corpus]))

In [ ]:
topic_indices[0]

In [ ]:
# Convert topic_indices to a Series, and convert the Series into a DataFrame
topic_indices_df = pd.DataFrame(pd.Series(topic_indices))

In [ ]:
topic_indices_df.head()

In [ ]:
# Concatenate trimmed and topic_indices_df
trimmed_topics = pd.concat([trimmed, topic_indices_df], axis=1)

In [ ]:
trimmed_topics.head()

In [ ]:
# Rename the column in trimmed_topics
trimmed_topics = trimmed_topics.rename(columns={0: 'topic_indices'})

In [ ]:
trimmed_topics.head()

In [ ]:
lda_model.print_topics()

In [ ]:
def get_topic_names_from_index(idx):
    if idx == 0:
        return ['Customer Service', 'Technology', 'Business']
    if idx == 1:
        return ['Encryption', 'Technology', 'Digital Security', 'Computer Science']
    if idx == 2:
        return ['Programming', 'API', 'Java', 'Technology']
    if idx == 3:
        return ['Cloud', 'AWS', 'Amazon Web Services', 'Internet', 'Technology']
    if idx == 4:
        return ['Teams', 'People', 'Office']
    if idx == 5:
        return ['Mobiles', 'Technology', 'Android', 'IOS', 'Apple', 'Google']
    if idx == 6:
        return ['Cloud', 'AWS', 'Amazon Web Services', 'Internet']
    if idx == 7:
        return ['Cloud', 'AWS', 'Amazon Web Services', 'Internet']
    if idx == 8:
        return ['Cloud', 'AWS', 'Amazon Web Services', 'Internet']
    if idx == 9:
        return ['Cloud', 'AWS', 'Amazon Web Services', 'Internet']
    return []

def get_topic_names(row):
    topic_names = [get_topic_names_from_index(idx) for idx in row]
    flat = [item for sublist in topic_names for item in sublist]
    return list(set(flat))

In [ ]:
# Drop text_no_punc and text_no_punc_lower from trimmed_topics
trimmed_topics = trimmed_topics.drop(['text_no_punc', 'text_no_punc_lower'], axis=1)

In [ ]:
trimmed_topics.head()

In [ ]:
# Iterate over the topic_indices column in trimmed_topics and apply get_topic_names_from_index
trimmed_topics['topic_names'] = trimmed_topics['topic_indices'].apply(lambda x: get_topic_names(x))

In [ ]:
trimmed_topics.head()

In [ ]:
# Drop topic_indices from trimmed_topics
trimmed_topics = trimmed_topics.drop('topic_indices', axis=1)

In [ ]:
trimmed_topics.head()

In [ ]:
trimmed_topics['topic_names']

In [ ]:
# Add the topic_names column to content
content = pd.concat([content, trimmed_topics['topic_names']], axis=1)

In [ ]:
content.head()

In [ ]:
trimmed.shape

In [ ]:
y.shape

In [ ]:
# x.columns

In [ ]:
trimmed.shape

In [ ]:
dups = trimmed[trimmed.duplicated(subset=['title', 'text_description'])]

In [ ]:
dups.head(20)

Based on the user topics, the recommended articles change.

* At the start of the day, recommend 10 articles based on user interests
* Adjust user interests based on
    * Direct feedback
    * Clicks
    * Time spent on the article page (some threshold)
* When a new user is created, user will choose some topics
* New articles should be recommended based on collaborative and content-based filtering

### Topic modeling on articles